# More Pandas, Part 2

Let's bring back in our Austin Animal Shelter dataset.

In [ ]:
import numpy as np
import pandas as pd
import requests 
from src.student_caller import three_random_students
from src.student_list import student_list
url = 'https://data.austintexas.gov/resource/9t4d-g238.json'
response = requests.get(url)
animals = pd.DataFrame(response.json())
animals.head()

### Reshaping a DataFrame


Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful, but it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot()` or `.pivot_table()`.

In [ ]:
# Code from previous notebook to convert age string to days upon outcome

def age_to_days(age):
    
    '''
    params: age upon outcome of shelter animal. 
    A number followed by a unit of time 
    'NULL', 'days', 'month', 'months', 'week', 'weeks', 'year', 'years'
    
    returns: days old at outcome
    '''
    
    age_split = age.split(' ')
    
    if len(age_split)  == 1:
        return np.nan
    
    elif age_split[1] == 'days' :
        return int(age_split[0])
    
    elif age_split[1] in (['month' or 'months']):
        return int(age_split[0]) * 30
    
    elif age_split[1] in ['week' or 'weeks'] :
        return int(age_split[0]) * 7
    
    else:
        return int(age_split[0]) * 365
    
    
animals['days_upon_outcome'] = animals['age_upon_outcome'].apply(age_to_days)

In [ ]:
animals.groupby(by=['outcome_type', 'sex_upon_outcome']).agg(np.mean)

In [ ]:
animals.pivot_table(index='outcome_type', columns='sex_upon_outcome', aggfunc=np.mean)

### Methods for Combining DataFrames: .join(), .merge(), .concat()

Pandas has several methods to combine dataframes.  The first two, join and merge, and very similar.  Here is a nice [Stack Overflow](https://stackoverflow.com/questions/22676081/what-is-the-difference-between-join-and-merge-in-pandas) response about the differences.

#### .join()

Join, by default, combines two dataframes based on their **index**, and performs a **left join**.

![images](images/left_join.png)

In [ ]:
toy1 = pd.DataFrame([[63, 142], [33, 47], [70, 20]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

In [ ]:
toy1

In [ ]:
toy1.set_index('age').join(toy2.set_index('age'))

For more on this method, check out the [doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)!

#### .merge()

Merge performs a bit differently.  We have to provide specific indices to merge on.  It is more typing, but gives us more control. By default it performs an **inner join**.

![inner_join](images/inner_join.png)


In [ ]:
ds_chars = pd.read_csv('ds_chars.csv', index_col=0)
ds_chars

In [ ]:
states = pd.read_csv('states.csv', index_col=0)
states

In [ ]:
ds_chars.merge(states, left_on='home_state', right_on='state', how='inner')

#### pd.concat()

This method takes a *list* of pandas objects as arguments.
It essentially pastes two dataframes together in the order that it encounters the records.

N.B. The cell below may produce a **Deprecation Warning**.

In [ ]:
ds_full = pd.concat([ds_chars, states])
ds_full

`pd.concat()`––and many other pandas operations––make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

### Making Use of Categories: One-Hot Encoding

OHE is a data transformation which will become much more important when we are building models. It sometimes is a bit tricky to get our heads around, so we will introduce it now.

One hot encoding takes as input a categorical feature, such as animal_type. It then uses the values of feature as headers of columns, and inserts 1 as a value in the column which represents the rows true category.  


In [ ]:
pd.get_dummies(animals['animal_type'])

get_dummies has a key parameter, `drop_first`, which drops the first column.

In [ ]:
pd.get_dummies(animals['animal_type'], drop_first=True)

Knowledge check: Although we dropped the Bird column, we can still tell which records are birds.  How?

In [ ]:
three_random_students(student_list)

If however we're in a later stage of the process and we're interested, say, in preparing a data pipeline, `pandas.get_dummies()` will prove inferior to other tools.

In practice, we will **not** use `pandas.get_dummies()`. The library Scikit-Learn (`sklearn`, included with your Anaconda installation) has a `OneHotEncoder` class that creates an object that persists. This makes it much more apt for production environments, and so it's good to get in the habit of using it.

Ultimately, we will use **many** tools from sklearn.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder()

In [ ]:
ohe.fit(animals[['animal_type']])

Now that the `OneHotEncoder` has been fitted to our data, it has newly available attributes and methods. In particular, it has access to the different categories that we're replacing:

In [ ]:
ohe.get_feature_names()

We'll have much more to say about `sklearn` syntax and about Python's object structure. But let's now transform our data to see what the new table looks like:

In [ ]:
ohe.transform(animals[['animal_type']])

For the sake of saving storage space, the return is a **sparse matrix**, but we can "re-inflate it if we want to see it in tabular form:

In [ ]:
types_encoded = ohe.transform(animals[['animal_type']]).todense()
types_encoded

Let's put it into a DataFrame:

In [ ]:
pd.DataFrame(types_encoded, columns=ohe.get_feature_names()).head()